In [ ]:
import sys
import inspect
from IPython.display import Markdown

# Add the parent directory to sys.path so 'src' can be imported
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Contextual fraction of 2-qudit states

This python project is for computing the contextual fraction of 2-qudit states with respect to Heisenberg Weyl operators. \


## Heisenberg-Weyl operators
For odd prime dimension $d$, the single qudit Pauli operators are defined by

\begin{equation*}
 X \ket{j} = \ket{j + 1} \quad \text{and} \quad Z \ket{j} = \omega^j \ket{j}
\end{equation*}

where $\omega = e^{2\pi i/d}$ and the addition is modulo $d$. The Heisenberg-Weyl group is generated by the Pauli operators $X$ and $Z$. 

>An element of the 2-qudit Heisenberg-Weyl group is specified by a symplectic vector $v=[p_1,q_1,p_2,q_2] \in \mathbb{Z}_3^4$: 
>\begin{equation*}
 W(v) = W([p_1,q_1,p_2,q_2]) = \omega^{2(p_1q_q+p_2q_2)}X^{p_1} Z^{q_1} \otimes X^{p_2}Z^{q_2}
\end{equation*}

where we have chosen the phase factor such that

\begin{equation*}
 W(v)W(v') = W(v + v'). 
\end{equation*}

For a given vector $v \in \mathbb{Z}_3^4$, we can generate the corresponding Heisenberg-Weyl operator using the function `src/utils/operators.pauli`. 

In [ ]:
import src.utils.operators as operators
Markdown(f"```python\n{inspect.getsource(operators)}\n```")

## Qudit contexts
A context is a set of pairwise commuting observables. We want to identify the maximal contexts of 2-qudit Heisenberg-Weyl operators. 
The operators $W(v)$ and $W(v')$ commute if the vectors $v$ and $v'$ are orthogonal with respect to the symplectic inner product:
\begin{equation*}
  [v, v'] = \sum_{i=1}^n (p_i q'_i - p'_i q_i) = 0. 
\end{equation*}

To identify maximal contexts, we need to identify maximal isotropic subspaces (i.e, sets of pairwise orthogonal vectors) of the symplectic vector space $\mathbb{Z}_3^4$.
Note that if $[v_1, v_2] = 0$, then $[(pv_1 + qv_2), (rv_1 + sv_2)] = 0$ for all $p,q,r,s \in \mathbb{Z}_3$. Hence, each context $C \in \mathcal{C}$ can be generated by $2$ commuting operators:
\begin{align*}
  \nonumber  && C  = \langle W(v_1), W(v_2) \rangle \\ 
   \nonumber &&= \{W(pv_1 + qv_2) \mid \forall p,q\in \mathbb{Z}_3 \}.
\end{align*}
where $[v_1, v_2] = 0$ and $v_2 \neq zv_1$ for any $z \in \mathbb{Z}_3$ (i.e, $v_1$ and $v_2$ are linearly independent). Each context contains $3^2 = 9$ operators (one is the identity operator). There are 40 such contexts for the 2-qutrit Heisenberg-Weyl group. The generators of the contexts are given by the symplectic vectors in `src/utils/contexts.py`.

In [ ]:
import src.utils.contexts as contexts
Markdown(f"```python\n{inspect.getsource(contexts)}\n```")

With the pairs of orthogonal vectors defined in `src/utils/contexts.py`, any context can be generated of the form: 
\begin{equation*}
  C = \langle A(c), B(c) \rangle = \{W(pA(c) + qB(c)) \mid \forall p,q\in \mathbb{Z}_3 \}
\end{equation*}

## Empirical model
To compute the contextual fraction, we need the measurement statistics of joint measurements of each context. For a given quantum state $\rho$, the empirical model is the measurement statistics of joint outcomes indexed by the contexts. For example, for the CHSH scenario, a possible empirical model is
\begin{equation*}
     \begin{array}{c|cccc} 
         & (0,0) & (1,0) & (0,1) & (1,1) \\
         \hline(A, B) & 1 / 2 & 0 & 0 & 1 / 2 \\
         \left(A, B^{\prime}\right) & 3 / 8 & 1 / 8 & 1 / 8 & 3 / 8 \\
         \left(A^{\prime}, B\right) & 3 / 8 & 1 / 8 & 1 / 8 & 3 / 8 \\
         \left(A^{\prime}, B^{\prime}\right) & 1 / 8 & 3 / 8 & 3 / 8 & 1 / 8
     \end{array}
\end{equation*}
Each row corresponds to a context, and each column corresponds to an even - a joint outcome of measurement of all the operators in the context.


For 2-qutrit Heisenberg-Weyl operators, the outcome of a measurement is of the form $\omega^a$ where $\omega = e^{2\pi i / 3}$ and $a \in \mathbb{Z}_3$. 
We label the outcome $\omega^a$ as $a$. 
Note that while there are 8 operators in each context (ignoring the identity operator), there are **not** $3^8 = 6561$ possible outcomes. 
The operators of a context share a common eigenbasis. 


>For 2-qutrits, there are 9 common eigenvectors, and hence there are only 9 possible outcomes for each context. 
This can also be seen from the fact that each context is generated by 2 operators, and each operator has 3 possible outcomes. 
The outcomes of the two generators completely determine the outcomes of all the other operators in the context.
Hence, we can label the outcomes of a context by $(a,b)$ where $a,b \in \mathbb{Z}_3$ are the outcomes of the two generators of the context. 

For the joint outcome $(a,b)$, the outcome of any operator $W(pA + qB)$ in the context is $\omega^{ap + bq}$.
To compute the probabilities of the joint outcomes, we need to construct projectors onto the common eigenvectors of the operators for each context.
 This can be done as follows. 
By spectral decomposition, we can write any Heisenberg-Weyl $W$ as 
\begin{equation*}
  W = \sum_{a \in \mathbb{Z}_3} \omega^a \Pi_W^a
\end{equation*}
where $\Pi_W^a$ is the projector onto the eigenspace of $W$ with eigenvalue $\omega^a$. 
From the above, we get the form of the projector $\Pi_W^a$ as
\begin{equation*}
  \Pi_W^a = \frac{1}{3} \sum_{j\in \mathbb{Z}_3} \omega^{-aj} W^j
\end{equation*}
With this, we can construct the projectors for the joint outcomes for a context:
\begin{equation*}
  \Pi_C^r = \frac{1}{|C|} \sum_{W \in C} \omega^{-r_W} W
\end{equation*}
where $r_W$ is the outcome of the operator $W$ for the joint outcome $r$.

>Hence, for context $C = \langle A(c), B(c) \rangle$, the projector onto the joint outcome $(a,b)$ is given by
\begin{equation*}
  \Pi_C^{(a,b)} = \frac{1}{9} \sum_{p,q \in \mathbb{Z}_3} \omega^{-ap - bq} W(pA(c) + qB(c))
\end{equation*}

The projector for the context $c$ and the joint outcome $(a,b)$ can be computed using the function `src/utils.measurements.projector`:

In [ ]:
from src.utils.measurements import projector
Markdown(f"```python\n{inspect.getsource(projector)}\n```")

With the projectors defined, we can compute the empirical model for a quantum state $\rho$. We generate the empirical model in a vectorized form, i.e., the empirical model is a vector of probabilities where the first $9$ entries correspond to the probabilities of the joint outcomes of the first context, the next $9$ entries correspond to the probabilities of the joint outcomes of the second context, and so on. 

For the outcome $(a,b)$, we can interpret $(a,b)$ as the ternary representation of the index of the outcome.
Hence,within a given context $c$, the outcome $(a,b)$ corresponds to the entry at index $[9c + (3 a + b)]$ in the empirical model vector. For 40 contexts, the empirical model is a vector of length $9 \times 40 = 360$. The empirical model for a given state can be computed using the function `src/utils.measurements.empirical_model`:

In [ ]:
from src.utils.measurements import empirical_model
Markdown(f"```python\n{inspect.getsource(empirical_model)}\n```")

## (Non)contextual fraction
Let's say we have a global value assignment $g$ for every Heisenberg-Weyl operator. 
That is, for each operator $W$, we know the measurement outcome $r_W = g(W) \in \mathbb{Z}_3$ deterministically. 
In the empirical model generated by this global assignment $g$, the probability of the joint outcome $r$ for a context $C$ is given by
\begin{equation*}
  p_C^g(r) = \begin{cases}
    1 & \text{if } r_W = g(W) \text{ for all } W \in C \\
    0 & \text{otherwise}
  \end{cases}
\end{equation*}

That is, each context has the probability $1$ for the joint outcome that is consistent with the global assignment $g$ and probability $0$ for all other joint outcomes.

For example, in the CHSH scenario, a global assignment $g$ could be:
\begin{equation*}
  g(A) = 0, g(A') = 1, g(B) = 0, g(B') = 1
\end{equation*}
The empirical model generated by this global assignment is:
\begin{equation*}
     \begin{array}{c|cccc}
         & (0,0) & (0,1) & (1,0) & (1,1) \\
         \hline(A, B) & 1 & 0 & 0 & 0 \\
         \left(A, B^{\prime}\right) & 0 & 1 & 0 & 0 \\
         \left(A^{\prime}, B\right) & 0 & 0 & 1 & 0 \\
         \left(A^{\prime}, B^{\prime}\right) & 0 & 0 & 0 & 1
     \end{array}
\end{equation*}

> A convex combination of empirical models is also a valid empirical model. An empirical model is said to be **noncontextual** if it can be expressed as a convex combination of empirical models generated by global assignments.

Any empirical model e can be decomposed into the form
\begin{equation*}
  e = \lambda e_{NC} + (1 - \lambda) e'
\end{equation*}
where $e_{NC}$ is a noncontextual empirical model. Note that this decomposition is not unique. By doing this, we explained a fraction of the events in terms of probabilistic combinations of global assignments.

Consider the convex decomposition where the weight of the noncontextual empirical model is maximized:
\begin{equation*}
  e = \lambda_{max} e_{NC} + (1 - \lambda_{max}) e'
\end{equation*}

For a decomposition of the above form, the 'residual' empirical model $e'$ is (strongly) contextual.

> This $\lambda_{max}$ is called the **noncontextual fraction** $NCF(e)$ of the empirical model $e$, and $1 - \lambda_{max}$ is called the **contextual fraction** $CF(e)$.

The empirical model $e$ admits the decomposition
\begin{equation*}
  e = NCFe_{NC} + (1 - NCF)e_{SC}
\end{equation*}

> An empirical model is said to be **contextual** if its contextual fraction is greater than $0$.

## Linear program for contextual fraction
Let $v^e$ be the vectorized form of an empirical model $e$.
Let $v^g$ be the vectorized form of an empirical model generated by a global assignment $g$. 
Define incidence matrix $M$ as the matrix whose columns are the vectorized forms of the empirical models generated by global assignments.